# Aula 3 - Parte Prática - Redução de Variância e Função Valor

## Introdução

Nesse terceiro notebook vamos aprender ...

### Objetivos:

- 

### Imports

> **Atenção:** não se esqueça de executar todos os `imports` necessários antes prosseguir com o tutorial.

In [ ]:
import gym
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from utils.agent import RLAgent
from utils.memory import OnPolicyReplay
from utils.networks import build_discrete_policy, build_continuous_policy, build_value_network
from utils.runner import train, evaluate
from utils.viz import plot_returns, plot_action_distribution


tf.get_logger().setLevel('ERROR') # ignore TensorFlow warnings

## 0. Configurações 

In [ ]:
# CartPole-v0, CartPole-v1
env = gym.make("CartPole-v0")
print(env.spec.reward_threshold)

In [ ]:
config = {
    # policy network
    "hidden_layers": [64, 64],
    "activation": "relu",

    # optimization
    "optimizer": "adam",
    "learning_rate": 1e-3,

    # training
    "train_batch_size": 3000,
}

## 1. REINFORCE (Vanilla Policy Gradients) 

In [ ]:
class REINFORCE(RLAgent):
    
    def __init__(self, obs_space, action_space, config, postprocessing=None):
        super(REINFORCE, self).__init__(obs_space, action_space, config)
        
        self.memory = OnPolicyReplay()
        self.policy = build_discrete_policy(self.obs_space, self.action_space, config["hidden_layers"], config["activation"])
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])
        
        self.postprocessing = postprocessing
        if postprocessing is None:
            self.postprocessing = np.sum

    def act(self, obs):
        return self._act(obs).numpy()
    
    @tf.function
    def _act(self, obs):
        action_dist = self.policy(obs[None,:])
        return action_dist.sample()[0]

    def observe(self, obs, action, reward, next_obs, done):
        self.memory.update(obs, action, reward, next_obs, done)

    def learn(self):
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()

        with tf.GradientTape() as tape:
            loss = self._loss_pg_fn(batch)
            gradients = tape.gradient(loss, self.policy.trainable_weights)

        self.optimizer.apply_gradients(zip(gradients, self.policy.trainable_weights))
      
        return loss

    def _loss_pg_fn(self, batch):
        states, actions, rewards = batch["states"], batch["actions"], batch["rewards"]
        
        n_episodes = len(rewards)

        loss = 0.0
        for episode in range(n_episodes):
            action_dist = self.policy(states[episode])
            log_prob = action_dist.log_prob(actions[episode])
            G = self.postprocessing(rewards[episode])
            loss += - tf.reduce_sum(log_prob * G)

        loss /= n_episodes
            
        return loss

In [ ]:
agent = REINFORCE(env.observation_space, env.action_space, config)

In [ ]:
total_timesteps = 1_000_000
timesteps, total_rewards, avg_total_rewards = train(agent, env, total_timesteps)

In [ ]:
plot_returns(timesteps, total_rewards, avg_total_rewards)

## 2. Policy Gradients: ignorando recompensas passadas

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left [ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(\mathbf{a}_t|\mathbf{s}_t) \hat{R}_t \right ]
$$

$$
\hat{R}_t = \sum_{t'=t}^T r_{t'}
$$


### 2.1 Reward-to-Go 

In [ ]:
def compute_reward_to_go(rewards):
    returns = []
    total_reward = 0.0

    for i in range(len(rewards) - 1, -1, -1):
        reward = rewards[i]
        total_reward += reward
        returns.append(total_reward)

    returns = returns[::-1]

    return np.array(returns)


In [ ]:
episode_length = 20
rewards = np.random.uniform(size=(episode_length,))

returns = compute_reward_to_go(rewards)

assert len(returns) == episode_length
assert np.allclose(np.cumsum(rewards[::-1])[::-1], returns)

### 2.2 REINFORCE + Reward-to-Go

In [ ]:
agent = REINFORCE(env.observation_space, env.action_space, config=config, postprocessing=compute_reward_to_go)

In [ ]:
total_timesteps = 1_000_000
timesteps, total_rewards, avg_total_rewards = train(agent, env, total_timesteps)

In [ ]:
plot_returns(timesteps, total_rewards, avg_total_rewards)

In [ ]:
# n_episodes = 5
# _ = evaluate(agent, env, n_episodes, render=True)

## 3. Policy Gradients: adicionando referências para os retornos (baseline)

###  3.1 Normalização dos *reward-to-go*

In [ ]:
def compute_normalized_reward_to_go(rewards):
    reward_to_go = compute_reward_to_go(rewards)
    mean, std = np.mean(reward_to_go), np.std(reward_to_go)
    reward_to_go = (reward_to_go - mean) / (std if std > 0.0 else 1.0)
    return reward_to_go


In [ ]:
agent = REINFORCE(env.observation_space, env.action_space, config, postprocessing=compute_normalized_reward_to_go)

In [ ]:
total_timesteps = 1_000_000
timesteps, total_rewards, avg_total_rewards = train(agent, env, total_timesteps)

In [ ]:
plot_returns(timesteps, total_rewards, avg_total_rewards)

In [ ]:
# n_episodes = 5
# _ = evaluate(agent, env, n_episodes, render=True)

### 3.2 Função Valor 

In [ ]:
class VPG(REINFORCE):
    
    def __init__(self, obs_space, action_space, config, postprocessing=None):
        super(VPG, self).__init__(obs_space, action_space, config, postprocessing)
        
        config = config["value_fn"]
        self.value_fn = build_value_network(obs_space, config["hidden_layers"], config["activation"])
        self.value_fn.compile(config["optimizer"], loss="MSE")
    
    def learn(self):
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()

        value_loss = self._train_value_fn(batch)
        policy_loss = self._train_policy_fn(batch)
      
        return policy_loss, value_loss
    
    def _train_policy_fn(self, batch):
        with tf.GradientTape() as tape:
            policy_loss = self._loss_pg_fn(batch)
            gradients = tape.gradient(policy_loss, self.policy.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.policy.trainable_weights))
        return policy_loss
    
    def _train_value_fn(self, batch):
        states, rewards = batch["states"], batch["rewards"]
        G = list(map(self.postprocessing, rewards))
        
        states = np.concatenate(states, axis=0)
        G = np.concatenate(G, axis=0)

        self.value_fn.fit(states, G, epochs=self.config["value_fn"]["epochs"], verbose=0)

        value_loss = self.value_fn.evaluate(states, G, verbose=0)
        return value_loss
    
    def _loss_pg_fn(self, batch):
        states, actions, rewards = batch["states"], batch["actions"], batch["rewards"]
        n_episodes = len(states)

        loss = 0.0
        for episode in range(n_episodes):
            action_dist = self.policy(states[episode])
            log_prob = action_dist.log_prob(actions[episode])

            G = self.postprocessing(rewards[episode])
            baseline = self.value_fn(states[episode])

            loss += - tf.reduce_sum(log_prob * (G - baseline))

        loss /= n_episodes
            
        return loss


In [ ]:
config = {
    **config,
    "value_fn": {
        # network
        "hidden_layers": [32, 32],
        "activation": "relu",
        
        # optimization
        "optimizer": "adam",
        "learning_rate": 3e-3,
        "epochs": 5,
    }
}

agent = VPG(env.observation_space, env.action_space, config, postprocessing=compute_normalized_reward_to_go)

In [ ]:
total_timesteps = 1_000_000
timesteps, total_rewards, avg_total_rewards = train(agent, env, total_timesteps)

In [ ]:
plot_returns(timesteps, total_rewards, avg_total_rewards)

In [ ]:
# n_episodes = 5
# _ = evaluate(agent, env, n_episodes, render=True)